James Caldwell
UVA IRA 10/25/24

This script was used to analyze international student deferrals and if Visa status might have an effect on deferral decisions.

In [2]:
import os
import pandas as pd

# Define the folder path
folder_path = r'V:\Data Requests\2024 Requests\Data Request 546\defer data'

# List to store the DataFrames
dataframes = {}

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.xlsx'):
        # Create the full file path
        file_path = os.path.join(folder_path, filename)
        # Load the Excel file into a DataFrame
        df = pd.read_excel(file_path)
        print(filename)
        print(len(df)-1)
        # Store the DataFrame in the dictionary with the filename (without extension) as the key
        dataframes[os.path.splitext(filename)[0]] = df

# Example to access a specific DataFrame
# df_example = dataframes['name_of_file_without_extension']



2018 Fall.xlsx
9
2019 Fall.xlsx
5
2020 Fall.xlsx
23
2020 Summer.xlsx
1
2021 Fall.xlsx
13
2022 Fall.xlsx
10
2023 Fall.xlsx
9
2024 Fall.xlsx
14


In [4]:
student_IDs = []

for file in dataframes.keys():
    student_IDs = student_IDs + dataframes[file]['Student System ID'].dropna().astype(int).tolist()


In [5]:
# Convert each element to a string and join them without commas
# print(" ".join(map(str, student_IDs))) 

In [6]:
# Combine each year's data into a single dataframe

all_data = pd.DataFrame()

for term_desc in dataframes.keys():

    # Drop rows with missing values in student system id
    df_here = dataframes[term_desc].dropna(subset=['Student System ID'])

    # Add column for defer year
    df_here['Defer Year'] = term_desc
    all_data = pd.concat([all_data, df_here], ignore_index=True)



C:\Users\ywe4kw\AppData\Local\Temp\ipykernel_11240\3207136594.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_here['Defer Year'] = term_desc
C:\Users\ywe4kw\AppData\Local\Temp\ipykernel_11240\3207136594.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_here['Defer Year'] = term_desc
C:\Users\ywe4kw\AppData\Local\Temp\ipykernel_11240\3207136594.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [7]:
# Load enrollment info
enrollment_data = pd.read_excel(r"V:\Data Requests\2024 Requests\Data Request 546\start dates\start dates.xlsx")

# Remove duplicates based on 'Student System ID', keeping the first instance
enrollment_data = enrollment_data.drop_duplicates(subset=['Student System ID'], keep='first')

enrollment_data.rename(columns={'Term Desc': 'Start Term Desc'}, inplace=True)

In [6]:
# Merge all_data and enrollment_data on 'Student System ID'
merged_data = pd.merge(all_data, enrollment_data, on='Student System ID', how='outer')

In [7]:
distinct_count = merged_data['Primary Name_y'].nunique()

print(distinct_count)

45


In [8]:
# merged_data['Delay Time (years)'] = merged_data['Start Term Desc'].iloc[:4].astype(int) - merged_data['Defer Year'].iloc[:4].astype(int)

In [13]:
# Drop rows with NaN values in the specified columns
cleaned_data = merged_data[['Start Term Desc', 'Defer Year']].dropna()



start_term_desc_chars = cleaned_data['Start Term Desc'].str[:4].astype(int)  # Convert to integers
defer_year_chars = cleaned_data['Defer Year'].str[:4].astype(int)  # Convert to integers

# Perform the subtraction for the valid rows
result = start_term_desc_chars - defer_year_chars

# Assign the calculated differences for the first four valid rows
merged_data.loc[cleaned_data.index, 'Defer Time'] = result




In [14]:
merged_data.to_excel(r'V:\Data Requests\2024 Requests\Data Request 546\summary.xlsx', index=False)